# Maximum Error in CLIRAD-LW's Cooling Rate Estimates  
The error here is the amount of deviation from the corresponding line-by-line value, so it can be either positive or negative.  For cooling rates, we show the error in the troposphere and the stratosphere.  The maximum is taken over the layers in the respective sphere (troposphere or stratosphere) and over the mid-latitude summer, sub-arctic winter and tropical atmosphere profiles.  

In [1]:
%reload_ext autoreload
%autoreload 2

import pymongo
import pandas as pd

from climatools.cliradlw.utils import *
from climatools.atm.absorbers import *
from climatools.atm.regions import *
from climatools.dataset import *

import pprint
from IPython import display

In [2]:
COMMITNUMBER = '1013f91'

In [3]:
def strat_top(): return .05
def atm_top(): return .01

In [4]:
client = pymongo.MongoClient('localhost', 27017)

In [19]:
MOLECULES = ('h2o', 'co2', 'o3', 'n2o', 'ch4')
BANDS = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)
ATMPROS = dict([('mls', 294), ('saw', 257), ('trp', 300)])
REGIONS = ('0.01', 'stra', 'trop')

In [6]:
def get_clirad_subparams(molecule=None, band=None):
    '''
    Return mls, saw and trp clirad parameter sub-dictionaries
    for a given molecule and clirad-lw spectral band.
    '''
    conc = nongreys_byband()[band][molecule]
    return {atmpro: {'commitnumber': COMMITNUMBER,
                     'band': [band],
                     'molecule': {molecule: conc},
                     'atmpro': atmpro,
                     'tsfc': tsfc}
            for atmpro, tsfc in ATMPROS.items()}

def to_lblnewparam(param=None):
    '''
    Convert clirad's input parameter dictionary 
    into something like lblnew's.
    '''
    band = param['band'][0]
    band = mapband_new2old()[band]
    atmpro = param['atmpro']
    tsfc = param['tsfc']
    
    p = {'band': band, 
         'atmpro': atmpro, 'tsfc': tsfc,
         'dv': .001, 'nv': 1000}
    
    if len(param['molecule']) > 1: p['molecule'] = param['molecule']
    else: 
        molecule, conc = list(param['molecule'].items())[0]
        p['molecule'] = molecule
        if conc == 'atmpro': p['conc'] = None
        else: p['conc'] = conc
    return p

def make_query(param=None):
    molecules = set(MOLECULES)
    q = {}
    for n, v in param.items():
        if n == 'molecule' and type(v) == dict:
            for mol, conc in v.items():
                q[f'param.molecule.{mol}'] = conc
            for mol in molecules - set(v.keys()):
                q[f'param.molecule.{mol}'] = {'$exists': 0}
        else:
            q[f'param.{n}'] = v
    return q

In [24]:
def get_maxerr_cool(molecule=None, band=None):
    '''
    Returns the maximum deviation from crd at the top and bottom
    of the atmosphere.
    '''
    tropos = tropopause_pressures()
    
    dclis = {atmpro: client.cliradnew.lw.find_one(make_query(param=p)) 
             for atmpro, p in get_clirad_subparams(molecule=molecule, band=band).items()}
    dclis = {atmpro: CliradnewLWModelData.from_mongodoc(doc) for atmpro, doc in dclis.items()}
    dclis = {atmpro: d.wgt_cool.sel(i=1).sum('band') for atmpro, d in dclis.items()}

    dcrds = {atmpro: client.lblnew.bestfit_lw.find_one(make_query(param=to_lblnewparam(p))) 
             for atmpro, p in get_clirad_subparams(molecule=molecule, band=band).items()}
    dcrds = {atmpro: LBLnewBestfitModelData.from_mongodoc(doc) for atmpro, doc in dcrds.items()}
    dcrds = {atmpro: d.crd_cool.sum('g') for atmpro, d in dcrds.items()}
    
    ddifs = {atmpro: dcli - dcrds[atmpro] for atmpro, dcli in dclis.items()}
    
    ddifs_tropo = [d.where(d.pressure > tropos[atmpro]) for atmpro, d in ddifs.items()]
    ddifs_strat = [d.where((tropos[atmpro] >= d.pressure) & (d.pressure > strat_top())) for atmpro, d in ddifs.items()]
    ddifs_topbt = [d.where((strat_top() >= d.pressure) & (d.pressure > atm_top())) for _, d in ddifs.items()]
    
    ddifs_tropo = [d.isel(pressure=abs(d.coolrg).argmax()) for d in ddifs_tropo]
    ddifs_strat = [d.isel(pressure=abs(d.coolrg).argmax()) for d in ddifs_strat]
    ddifs_topbt = [d.isel(pressure=abs(d.coolrg).argmax()) for d in ddifs_topbt]
    
    ddifs_tropo = sorted([d.coolrg for d in ddifs_tropo], key=lambda x: abs(x), reverse=True)
    ddifs_strat = sorted([d.coolrg for d in ddifs_strat], key=lambda x: abs(x), reverse=True)
    ddifs_topbt = sorted([d.coolrg for d in ddifs_topbt], key=lambda x: abs(x), reverse=True)

    return dict(zip(REGIONS, [float(d[0]) for d in (ddifs_topbt, ddifs_strat, ddifs_tropo)]))

In [28]:
def maxerr_cool_all():
    df = pd.DataFrame(index=MOLECULES, 
                      columns=pd.MultiIndex.from_product([BANDS, REGIONS], 
                                                         names=('band', 'region')))
    df.index.name = 'gas'

    for band, molconcs in nongreys_byband().items():
        for mol, conc in molconcs.items():
            for level, v in get_maxerr_cool(molecule=mol, band=band).items():
                df.loc[mol, (band, level)] = round(v, 4)

    df.fillna('', inplace=True)
    return df

In [36]:
df = maxerr_cool_all()
pd.set_option('display.max_columns', 500)
display.display(df)

display.display(display.Markdown((f'**0.01** region is the region between 0.05 mb and 0.01 mb.  '
                                  f'**stra** is the stratosphere, from the tropopause to 0.05 mb.  '
                                  f'**trop** is the troposphere, from the surface to the tropopause.  '
                                  f'The tropopause pressure is different for different atmosphere profiles.')))

band        1                       2                       3           \
region    0.01    stra    trop    0.01    stra    trop    0.01    stra   
gas                                                                      
h2o    -0.2291  0.0719  0.0566 -0.0867 -0.0218  0.0848 -0.0016  -0.002   
co2                                                    -0.1527 -0.0922   
o3                                                                       
n2o                                                     0.0042 -0.0046   
ch4                                                                      

band                4                       5                       6   \
region    trop    0.01    stra    trop    0.01    stra    trop    0.01   
gas                                                                      
h2o     0.0477 -0.0012 -0.0026  0.0504 -0.0002 -0.0008  0.0223      -0   
co2    -0.0082 -0.7342  0.7071 -0.0183 -0.1418  0.0889  0.0107 -0.0014   
o3                                                                       
n2o     0.0014                                                           
ch4                                                                      

band                        7                       8                   \
region    stra    trop    0.01    stra    trop    0.01    stra    trop   
gas                                                                      
h2o    -0.0002  0.0497      -0      -0   0.016 -0.0001 -0.0005  0.0248   
co2    -0.0057 -0.0005 -0.0016 -0.0062 -0.0003                           
o3                     -0.0909 -0.1814 -0.0089                           
n2o                                                                      
ch4                                                                      

band        9                       10                      11                  
region    0.01    stra    trop    0.01    stra    trop    0.01    stra    trop  
gas                                                                             
h2o     0.0037 -0.0075  0.0262 -0.0359 -0.0279 -0.0293  0.0007 -0.0015  0.0141  
co2                                                     0.0466  0.0602   0.003  
o3                                                                              
n2o     0.0241  0.0241 -0.0106                                                  
ch4     0.0397  0.0404 -0.0041

**0.01** region is the region between 0.05 mb and 0.01 mb.  **stra** is the stratosphere, from the tropopause to 0.05 mb.  **trop** is the troposphere, from the surface to the tropopause.  The tropopause pressure is different for different atmosphere profiles.

# Maximum Error in CLIRAD-LW's Flux Estimates  
The error here is the amount of deviation from the corresponding line-by-line value, so it can be either positive or negative.  For fluxes, we show the error in the upward flux at the top of the atmosphere and in the downward flux at the surface.  The maximum is taken over the mid-latitude summer, sub-arctic winter and tropical atmosphere profiles.

In [11]:
def get_maxerr_flux(molecule=None, band=None):
    '''
    Returns the maximum deviation from crd at the top and bottom
    of the atmosphere.
    '''
    tropos = tropopause_pressures()
    
    dclis = {atmpro: client.cliradnew.lw.find_one(make_query(param=p)) 
             for atmpro, p in get_clirad_subparams(molecule=molecule, band=band).items()}
    dclis = {atmpro: CliradnewLWModelData.from_mongodoc(doc) for atmpro, doc in dclis.items()}
    dclis = {atmpro: d.wgt_flux.sel(i=1).sum('band') for atmpro, d in dclis.items()}

    dcrds = {atmpro: client.lblnew.bestfit_lw.find_one(make_query(param=to_lblnewparam(p))) 
             for atmpro, p in get_clirad_subparams(molecule=molecule, band=band).items()}
    dcrds = {atmpro: LBLnewBestfitModelData.from_mongodoc(doc) for atmpro, doc in dcrds.items()}
    dcrds = {atmpro: d.crd_flux.sum('g') for atmpro, d in dcrds.items()}
    
    ddifs = {atmpro: dcli - dcrds[atmpro] for atmpro, dcli in dclis.items()}

    ddifs_toa = [float(d['flug'].isel(pressure=0).values) for atmpro, d in ddifs.items()]
    ddifs_sfc = [float(d['fldg'].isel(pressure=-1).values) for atmpro, d in ddifs.items()]
    
    maxdif_toa = sorted(ddifs_toa, key=lambda x: abs(x), reverse=True)[0]
    maxdif_sfc = sorted(ddifs_sfc, key=lambda x: abs(x), reverse=True)[0]
    return dict(TOA=maxdif_toa, SFC=maxdif_sfc)

In [30]:
def maxerr_flux_all():
    df = pd.DataFrame(index=MOLECULES, 
                      columns=pd.MultiIndex.from_product([BANDS, ('SFC', 'TOA')], 
                                                         names=('band', 'level')))
    df.index.name = 'gas'

    for band, molconcs in nongreys_byband().items():
        for mol, conc in molconcs.items():
            for level, v in get_maxerr_flux(molecule=mol, band=band).items():
                df.loc[mol, (band, level)] = round(v, 4)

    df.fillna('', inplace=True)
    return df

In [31]:
df = maxerr_flux_all()

In [33]:
#pd.set_option('display.max_columns', 50)
display.display(df)

band       1               2               3               4               5   \
level     SFC     TOA     SFC     TOA     SFC     TOA     SFC     TOA     SFC   
gas                                                                             
h2o   -0.0026  0.2108  0.3749  0.3224  0.4148 -0.0979 -0.3591 -0.1034  0.2303   
co2                                    0.2371  -0.121  0.0979  0.4181  0.4871   
o3                                                                              
n2o                                    0.0183 -0.0554                           
ch4                                                                             

band               6               7               8               9           \
level     TOA     SFC     TOA     SFC     TOA     SFC     TOA     SFC     TOA   
gas                                                                             
h2o   -0.0618  0.4709 -0.0394  0.0833  -0.041 -0.1404 -0.1106  -0.131   -0.32   
co2    0.3676 -0.0679 -0.0449  -0.054 -0.0372                                   
o3                            -0.1392  0.4044                                   
n2o                                                           -0.0549  0.2212   
ch4                                                           -0.2725 -0.0316   

band       10              11          
level     SFC     TOA     SFC     TOA  
gas                                    
h2o    0.0361  0.1406  0.5221   0.371  
co2                    0.1651  0.0999  
o3                                     
n2o                                    
ch4

# --

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')